# **Unit 1 - Progam 1 - Expense Management**

## **Main Program**

## Class Definition

In [1]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, clear_output
from datetime import date
from ipyfilechooser import FileChooser 
import time
import json
import sys

class ExpenseManager:
    def __init__(self):
        self.expenses_list = []
        self.current_expense_date = ""
        self.current_expense_category = ""
        self.current_expense_amount = 0.0
        self.current_expense_description = ""
        self.menu_container = None
        self.expenses_input_form = None
        self.expenses_save_form = None
        self.expenses_load_form = None
        self.expenses_view_form = None
        self.budget_tracker_form = None
        self.success_color = "#4CAF50"
        self.error_color = "#F44336"

        # Create title
        self.ExpenseManagerBanner = widgets.HTML(
            value="""
            <div style='text-align: center; padding: 20px; background-color: #2196F3; color: white; border-radius: 10px;'>
                <h1>EXPENSE MANAGER</h1>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

    #
    # Display Application Menu
    #
    def DisplayMenu(self):

        # Create menu buttons
        add_expense_btn = widgets.Button(
            description='1. Add Expense',
            button_style='primary',
            layout=widgets.Layout(width='300px', height='50px')
        )
        
        view_expenses_btn = widgets.Button(
            description='2. View Expenses',
            button_style='info',
            layout=widgets.Layout(width='300px', height='50px')
        )
        
        track_budget_btn = widgets.Button(
            description='3. Track Budget',
            button_style='warning',
            layout=widgets.Layout(width='300px', height='50px')
        )
        
        save_expenses_btn = widgets.Button(
            description='4. Save Expenses To File',
            button_style='success',
            layout=widgets.Layout(width='300px', height='50px')
        )

        load_expenses_btn = widgets.Button(
            description='5. Load Expenses From File',
            button_style='',
            layout=widgets.Layout(width='300px', height='50px')
        )

        # Custom color
        load_expenses_btn.style.button_color = '#9C27B0'  # Purple
        load_expenses_btn.style.text_color = 'white'

        
        exit_btn = widgets.Button(
            description='5. Exit Application',
            button_style='danger',
            layout=widgets.Layout(width='300px', height='50px')
        )
        
        # Button click handlers
        def on_add_expense(b):
            self.PromptForExpenses()
        
        def on_view_expenses(b):
            self.ViewExpenses()
        
        def on_track_budget(b):
            self.TrackBudget()
        
        def on_save_expenses(b):
            self.SaveExpensesToFile()
            
        def on_load_expenses(b):
            self.LoadExpensesFromFile()
        
        def on_exit(b):
            with self.shared_output:
                clear_output(wait=True)
            print("Thank you for using Expense Manager!")
            print("Application closed.")
    
            # Close the container
            self.menu_container.close()
        
        # Attach handlers
        add_expense_btn.on_click(on_add_expense)
        view_expenses_btn.on_click(on_view_expenses)
        track_budget_btn.on_click(on_track_budget)
        save_expenses_btn.on_click(on_save_expenses)
        load_expenses_btn.on_click(on_load_expenses)
        exit_btn.on_click(on_exit)
        
        # Create menu container
        self.menu_container = widgets.VBox([ 
            self.ExpenseManagerBanner,
            widgets.HTML(value="<br>"),
            add_expense_btn,
            view_expenses_btn,
            track_budget_btn,
            save_expenses_btn,
            load_expenses_btn,
            exit_btn,
            widgets.HTML(value="<br>")
        ], layout=widgets.Layout(padding='20px'))
        
        # Create shared output area FIRST
        self.shared_output = widgets.Output()

        with self.shared_output:
            display(self.menu_container)
        
        # Add this instead:
        display(self.shared_output)  # Display AFTER populating it

    #
    # Prompt for Expenses
    #    
    def PromptForExpenses(self):

        print(f"About to clear and display")
        prompt_for_expenses_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #00ACC1; color: white; border-radius: 10px;'>
                <h3>ENTER EXPENSE DATA</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        # Create input widgets
        date_picker = widgets.DatePicker(
                        description='Select a Date',
                        value=date.today(),
                        disabled=False)
        
        category_input = widgets.Text(
            description='Category:',
            placeholder='e.g., Food, Transport'
        )
        
        amount_input = widgets.FloatText(
            description='Amount:',
            value=0.0,
            min=0.0,
            max=999999.99,
            step=.01
        )

        def validate_amount(change):
            if change['new'] <= 0.00:
                amount_input.value = 0.00

        amount_input.observe(validate_amount, names='value')

        # Formatted currency display
        amount_display = widgets.HTML(
            value="<span style='font-size: 16px; font-weight: bold; color: #2e7d32;'>$0.01</span>")
        
    
        def format_currency(change):
            formatted = f"${change['new']:,.2f}"
            amount_display.value = f"<span style='font-size: 16px; font-weight: bold; color: #2e7d32;'>{formatted}</span>"
    
        amount_input.observe(format_currency, names='value')
        
        description_input = widgets.Textarea(
            description='Description:',
            placeholder='Enter Expense Details...'
        )

        add_expenses_button = widgets.Button(
            description='Add Expense',
            button_style='success'
        )

        # Validation Errors
        validation_label = widgets.HTML(value="")

        def on_save(b):
            # Clear previous error
            validation_label.value = ""
            
            # Validate inputs
            errors = []
            
            if not date_picker.value:
                errors.append("Date is required")
            
            if not category_input.value.strip():
                errors.append("Category is required")
            
            if amount_input.value <= 0:
                errors.append("Amount must be greater than 0")
            
            if not description_input.value.strip():
                errors.append("Description is required")
            
            # If there are errors, show them
            if errors:
                validation_label.value = f"<span style='color: {self.error_color};'><b>Errors:</b><br>{'<br>'.join(errors)}</span>"
                return
            
            # All valid - save the expense
            self.current_expense_date = date_picker.value.strftime('%Y-%m-%d')
            self.current_expense_category = category_input.value
            self.current_expense_amount = amount_input.value
            self.current_expense_description = description_input.value

            self.expenses_list.append({"ExpenseDate" : self.current_expense_date,
                                       "Category": self.current_expense_category,
                                       "Amount" : self.current_expense_amount,
                                       "Description" : self.current_expense_description})
            
            # Success message
            validation_label.value = f"<span style='color: {self.success_color};'><b>Expense added:</b> ${self.current_expense_amount:.2f} on {self.current_expense_date}</span>"

            # Optionally clear the form
            category_input.value = ""
            amount_input.value = 0.0
            description_input.value = ""

        add_expenses_button.on_click(on_save)

        # Back button (Cancel)
        back_button = widgets.Button(
            description='← Back to Main Menu',
            button_style='',
            layout=widgets.Layout(width='150px')
        )
        def on_back(b):
            with self.shared_output:
                validation_label.value = f"<span style='color: {self.success_color};'><b>Returning to Main Expenses Menu</b></span>"
                clear_output(wait=True)  # This clears the form
                display(self.menu_container)

        back_button.on_click(on_back)

        # Container/Dialog to hold all widgets
        self.expenses_input_form = widgets.VBox([
            self.ExpenseManagerBanner,
            prompt_for_expenses_header,
            widgets.HTML(value="<div style='margin: 5px;'></div>"),
            date_picker,
            category_input,
            amount_input,
            description_input,
            widgets.HTML(value="<br>"),
            add_expenses_button,
            back_button,
            validation_label
        ], layout=widgets.Layout(padding='20px'))

        self.shared_output.clear_output(wait=False)
        with self.shared_output:
            display(self.expenses_input_form)

    #
    # View All Expenses (currently loaded in memory)
    #  
    def ExpensesViewer(self, invalid_expenses=None):

        valid_expenses_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #81C784; color: white; border-radius: 10px;'>
                <h3>VALID EXPENSES</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        # Create Output widget to hold the DataFrame
        valid_expenses_dataframe = pd.DataFrame(self.expenses_list)
        valid_expenses_output = widgets.Output(border='1px solid #ddd')
        with valid_expenses_output:
            display(valid_expenses_dataframe)

        scrollable_valid_expenses_container = widgets.VBox([valid_expenses_output], 
                                                           layout=widgets.Layout( height='200px', 
                                                                                 width='700px',
                                                                                 overflow_x='auto', 
                                                                                 overflow_y='auto', 
                                                                                 border='1px solid #ddd'))            

        child_widgets = [
            valid_expenses_header,
            scrollable_valid_expenses_container
        ]

        if not self.expenses_list:
            validation_label= widgets.HTML(value=f"<span style='color: {self.error_color};'><b>There are no valid expenses loaded to be displayed</b></span>")
            child_widgets.append(widgets.HTML(value="<div style='margin: 5px;'></div>"))
            child_widgets.append(validation_label)

        if invalid_expenses:
            
            invalid_expenses_header = widgets.HTML(
                value="""
                <div style='text-align: center; background-color: #FF6B6B; color: white; border-radius: 10px;'>
                    <h3>INVALID EXPENSES</h3>
                </div>
                """,
                layout=widgets.Layout(width='400px')
            )

            invalid_expenses_dataframe = pd.DataFrame(invalid_expenses)
            invalid_expenses_output = widgets.Output()
            with invalid_expenses_output:
                display(invalid_expenses_dataframe)

            scrollable_invalid_expenses_container = widgets.VBox([invalid_expenses_output], 
                                                                layout=widgets.Layout( height='200px', 
                                                                                       width='700px',
                                                                                       overflow_x='auto', 
                                                                                       overflow_y='auto', 
                                                                                       border='1px solid #ddd'))            

            child_widgets.append(widgets.HTML(value="<div style='margin: 5px;'></div>"))
            child_widgets.append(widgets.HTML(value="<div style='margin: 5px;'></div>"))
            child_widgets.append(invalid_expenses_header)
            child_widgets.append(scrollable_invalid_expenses_container)
            child_widgets.append(widgets.HTML(value="<div style='margin: 5px;'></div>"))

            invalid_expenses_warning_label = widgets.HTML(
                value=f"<span style='color: {self.error_color};'><b>Invalid Expenses were found with data validation errors. They will not be loaded.</b></span>",
                layout=widgets.Layout(width='500px')
            )
            child_widgets.append(invalid_expenses_warning_label)

        expenses_viewer_container = widgets.VBox(
            child_widgets, layout=widgets.Layout(padding='20px'))

        return expenses_viewer_container

    #
    # View All Expenses (currently loaded in memory)
    #    
    def ViewExpenses(self):

        expenses_viewer_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #00ACC1; color: white; border-radius: 10px;'>
                <h3>VIEW EXPENSES</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        expenses_viewer = self.ExpensesViewer()
       
        # Back button (Cancel)
        back_button = widgets.Button(
            description='← Back to Main Menu',
            button_style='',
            layout=widgets.Layout(width='150px')
        )

        def on_back(b):
            with self.shared_output:
                clear_output(wait=True)  # This clears the form
                display(self.menu_container)

        back_button.on_click(on_back)

        # Container/Dialog to hold all widgets
        self.expenses_view_form = widgets.VBox([
            self.ExpenseManagerBanner,
            expenses_viewer_header,
            expenses_viewer,
            widgets.HTML(value="<br>"),
            back_button,
        ], layout=widgets.Layout(padding='20px'))

        # Hide Main Menu and display Expense View Form
        with self.shared_output:
            clear_output(wait=True)
            display(self.expenses_view_form)

    #
    # Track Budget
    #
    def TrackBudget(self):

        budget_amount_remaining = 0.0
        total_expenses = sum(expense['Amount'] for expense in self.expenses_list)
        validation_label = widgets.HTML(value="")

        track_budget_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #00ACC1; color: white; border-radius: 10px;'>
                <h3>TRACK BUDGET</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        expenses_viewer = self.ExpensesViewer()

        budget_label = widgets.HTML(value="<b>Enter Budget Amount:</b>")
        budget_amount_input = widgets.FloatText(value=0.0,
                                                min=0.0,
                                                max=999999.99,
                                                step=.01,
                                                layout=widgets.Layout(width='150px')  
                                            )
        
        total_expenses_display = widgets.HTML(value=f"<b>Total Expenses:</b> ${total_expenses:,.2f}")
        budget_amount_remaining_display = widgets.HTML(value=f"<b>Budget Surplus:</b> ${budget_amount_remaining:,.2f}")

        budget_info_row = widgets.HBox([budget_label,
                                        budget_amount_input,
                                        widgets.HTML(value="<div style='width: 20px;'></div>"),  
                                        total_expenses_display,
                                        widgets.HTML(value="<div style='width: 20px;'></div>"),
                                        budget_amount_remaining_display])
        
        calculate_budget_button = widgets.Button(description='Calculate Budget',
                                                button_style='success',
                                                icon='check')
        
        def on_calculate_budget_button(b):
            
            budget_amount_remaining = budget_amount_input.value - total_expenses
            budget_state = ""
            budget_state_color = ""
            budget_state_message = ""

            if budget_amount_remaining > 0.0:
                budget_state = "Budget Surplus"
                budget_state_color = self.success_color
                budget_state_message = "Congratulations! You have a surplus budget."
            elif budget_amount_remaining == 0.0:
                budget_state = "Balanced Budget"
                budget_state_color = "#000000"
                budget_state_message = "OK! You have a balanced budget."
            else:
                budget_state = "Budget Deficit"
                budget_state_color = self.error_color
                budget_state_message = "Shame! You have negative cash flow."

            budget_amount_remaining_display.value = f"<b>{budget_state}:</b> <span style='color: {budget_state_color};'>${budget_amount_remaining:,.2f}</span>"   
            validation_label.value = f"<span style='color: {budget_state_color};'><b>{budget_state_message}</b></span>"

        calculate_budget_button.on_click(on_calculate_budget_button)

        # Back button (Cancel)
        back_button = widgets.Button(
            description='← Back to Main Menu',
            button_style='',
            layout=widgets.Layout(width='150px')
        )
        
        def on_back(b):
            with self.shared_output:
                clear_output(wait=True)  # This clears the form
                display(self.menu_container)

        back_button.on_click(on_back)

        # Disable calculation if we don't have any expenses
        if not self.expenses_list:
            calculate_budget_button.disabled = True
            validation_label.value = f"<span style='color: {self.error_color};'><b>Valid Expenses must be loaded before budget can be calculated</b></span>"
        else:
            calculate_budget_button.disabled = False
            validation_label.value = f"<span style='color: {self.success_color};'><b>Enter a budget value and click 'Calculate Budget'</b></span>"


        # Container/Dialog to hold all widgets
        self.budget_tracker_form = widgets.VBox([
            self.ExpenseManagerBanner,
            track_budget_header,
            widgets.HTML(value="<div style='margin: 5px;'></div>"),
            expenses_viewer,
            budget_info_row,
            calculate_budget_button,
            widgets.HTML(value="<br>"),
            back_button,
            validation_label
        ], layout=widgets.Layout(padding='20px'))

        # Hide Main Menu and display Expense Input Form
        with self.shared_output:
            clear_output(wait=True)
            display(self.budget_tracker_form)

    #
    # Save Expenses
    #
    def SaveExpensesToFile(self):

        expenses_save_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #00ACC1; color: white; border-radius: 10px;'>
                <h3>SAVE EXPENSES</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        # Create file chooser
        file_chooser = FileChooser(
            path='.',
            filename='ExpensesData.csv',
            title='<b>Save Expenses File as:</b>',
            show_hidden=False,
            select_default=True
        )

        save_button = widgets.Button(
            description='Save File',
            button_style='success',
            icon='check'
        )

        validation_label = widgets.HTML(value="")

        def on_save_clicked(b):
            with self.shared_output:
                clear_output(wait=True)  
                if file_chooser.selected:
                    try:
                        # If I want to switch to json format..
                        # with open(file_chooser.selected, 'w') as f:
                        #     json.dump(self.expenses_list, f, indent=4)

                        # Save to CSV
                        expenses_dataframe = pd.DataFrame(self.expenses_list)
                        expenses_dataframe.to_csv(file_chooser.selected, index=False, header=True)
                        validation_label.value = f"<span style='color: {self.success_color};'><b>✓ File saved successfully: </b> {file_chooser.selected}</span>"

                    except Exception as e:
                        validation_label.value = f"<span style='color: {self.error_color};'><b>✗ Error saving file:</b><br>{str(e)}</span>"
                else:
                    validation_label.value = f"<span style='color: {self.error_color};'><b>✗ No file selected</b></span>"

        save_button.on_click(on_save_clicked)

        # Back button (Cancel)
        back_button = widgets.Button(
            description='← Back to Main Menu',
            button_style='',
            layout=widgets.Layout(width='150px')
        )

        def on_back(b):
            with self.shared_output:
                clear_output(wait=True)  # This clears the form
                display(self.menu_container)

        back_button.on_click(on_back)        

        # Container/Dialog to hold all widgets
        self.expenses_save_form = widgets.VBox([
            self.ExpenseManagerBanner,
            expenses_save_header,
            widgets.HTML(value="<div style='margin: 5px;'></div>"),
            file_chooser,
            save_button,
            widgets.HTML(value="<br>"),
            back_button,
            validation_label 
        ], layout=widgets.Layout(padding='20px'))

        # Hide Main Menu and display Expenses Save Form
        with self.shared_output:
            clear_output(wait=True)
            display(self.expenses_save_form)

    #
    # Validate Date Format
    #
    def is_valid_date(self, date_string, date_format='%Y-%m-%d'):

        try:
            date.fromisoformat(date_string)
            return True
        except (ValueError, TypeError):
            return False
    #
    # Validate Expenses
    #
    def ValidateExpenses(self, expenses_list):

        valid_expenses = []
        invalid_expenses = []

        for expense in expenses_list:

            if expense:

                errors = []

                expenseDate = expense.get('ExpenseDate')  
                if not(expenseDate and isinstance(expenseDate, str) and expenseDate.strip()):
                    errors.append("ExpenseDate is required")
                else:
                    if not self.is_valid_date(expenseDate):
                        errors.append("ExpenseDate has Invalid Date Format")
                
                category = expense.get('Category')  # Returns None (no error!)
                if not(category and isinstance(category, str) and category.strip()):
                    errors.append("Category is required")
                
                amount = expense.get('Amount')  
                if not amount:
                    errors.append("Amount is required")
                else:
                    if not amount > 0:
                        errors.append("Amount must be greater than 0")
                
                description = expense.get('Description')  
                if not(description and isinstance(description, str) and description.strip()):
                    errors.append("Description is required")

                if not errors:
                    valid_expenses.append(expense)

                else:
                    expense['ValidationErrors'] = errors
                    invalid_expenses.append(expense)

        return valid_expenses, invalid_expenses

    #
    # Load Expenses
    #
    def LoadExpensesFromFile(self):

        invalid_expenses_list = []
        
        expenses_load_header = widgets.HTML(
            value="""
            <div style='text-align: center; background-color: #00ACC1; color: white; border-radius: 10px;'>
                <h3>LOAD EXPENSES</h3>
            </div>
            """,
            layout=widgets.Layout(width='400px')
        )

        # Create file chooser
        file_chooser = FileChooser(
            path='.',
            filename='ExpensesData.csv',
            title='<b>Load Expenses File From:</b>',
            show_hidden=False,
            select_default=True
        )

        load_button = widgets.Button(
            description='Load File',
            button_style='success',
            icon='check'
        )

        validation_label = widgets.HTML(value="")

        def on_load_clicked(b):
            with self.shared_output:
                clear_output(wait=True)  
                if file_chooser.selected:
                    try:

                        # if we want to load json files
                        # with open(file_chooser.selected, 'r') as f:
                        #     loaded_expenses = json.load(f)

                        # Load CSV into DataFrame and convert to list of dicts
                        imported_dataframe = pd.read_csv(file_chooser.selected)
                        loaded_expenses = imported_dataframe.to_dict('records')

                        self.expenses_list, invalid_expenses_list = self.ValidateExpenses(loaded_expenses)
                        
                        validation_label.value = f"<span style='color: {self.success_color};'><b>✓ File loaded successfully: </b> {file_chooser.selected}</span>"
                        expenses_viewer = self.ExpensesViewer(invalid_expenses_list)
                        
                        self.expenses_load_form.children = [
                            self.ExpenseManagerBanner,
                            expenses_load_header,
                            widgets.HTML(value="<div style='margin: 5px;'></div>"),
                            file_chooser,
                            load_button,
                            widgets.HTML(value="<div style='margin: 5px;'></div>"),
                            expenses_viewer,      
                            validation_label,
                            widgets.HTML(value="<br>"),
                            back_button
                        ]

                    except Exception as e:
                        validation_label.value = f"<span style='color: {self.error_color};'><b>✗ Error loading file:</b><br>{str(e)}</span>"
                else:
                    validation_label.value = f"<span style='color: {self.error_color};'><b>✗ No file selected</b></span>"

        load_button.on_click(on_load_clicked)
            
        # Back button (Cancel)
        back_button = widgets.Button(
            description='← Back to Main Menu',
            button_style='',
            layout=widgets.Layout(width='150px')
        )

        def on_back(b):
            with self.shared_output:
                clear_output(wait=True)  # This clears the form
                display(self.menu_container)

        back_button.on_click(on_back)        

        # Container/Dialog to hold all widgets
        self.expenses_load_form = widgets.VBox([
            self.ExpenseManagerBanner,
            expenses_load_header,
            widgets.HTML(value="<div style='margin: 5px;'></div>"),
            file_chooser,
            load_button,
            widgets.HTML(value="<div style='margin: 5px;'></div>"),
            back_button,
            validation_label
        ], layout=widgets.Layout(padding='20px'))

        # Hide Main Menu and display Expenses Load Form
        with self.shared_output:
            clear_output(wait=True)
            display(self.expenses_load_form)            



## **Tests**

In [3]:
# import sys
# print(sys.executable)
# print(sys.path)

test = ExpenseManager()
test.DisplayMenu()



Output()